In [4]:
### Illustration du biais de regularisation
### 26 septembre 2019
### Jeremy LHour

library("MASS")
library("glmnet")

# fonctions utilisateur
source("functions/DataSim.R") 

set.seed(12071990)
n = 2000
data = DataSim(n, p=20, Ry= .2, Intercept=F)
y = data$y; X = data$X

Warning message in c * gamma:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in c * b:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”

In [16]:
PostLasso <- function(X,y,nopen){
    #'@param nopen indices des variables a ne pas penaliser
    phi = rep(1,ncol(X)); phi[nopen] = 0
    cv.lasso = cv.glmnet(X,y, family="gaussian",alpha=1,penalty.factor=phi,nfolds=5)
    b.lasso = coef(cv.lasso); b.lasso = b.lasso[-1] # on enleve la constante
    s.hat = which(b.lasso != 0) # indices des variables actives parmi les X
    print(s.hat)
    postlasso = lm(y ~ X[,s.hat])
    b.pl = coef(postlasso)

    return(b.pl)
}

PostStudent <- function(X,y,noppen,alpha=.05){
    #'@param nopen indices des variables a ne pas penaliser
    #'@param alpha niveau de significativite pour selection
    fullreg = lm(y ~ X)
    s.hat = summary(fullreg)$coef[,"Pr(>|t|)"] < alpha
    s.hat = s.hat[-1]
    s.hat = union(s.hat,nopen)
    shortreg = lm(y ~ X[,s.hat])
    b.ps = coef(shortref)

    return(b.ps)
}


In [17]:
    fullreg = lm(y ~ X)
    s.hat = summary(fullreg)$coef[,"Pr(>|t|)"] < .05
    s.hat = s.hat[-1]
    s.hat = union(s.hat,nopen)
    shortreg = lm(y ~ X[,s.hat])
    b.ps = coef(shortref)
print(b.ps)

ERROR: Error in eval(expr, envir, enclos): object 'alpha' not found
